# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path("Resources/lending_data.csv")
df_lend = pd.read_csv(file_path)
df_lend.drop(["homeowner"], axis=1)


,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,53000,0.433962,5,1,23000,low_risk
...,...,...,...,...,...,...,...,...
77531,19100.0,11.261,86600,0.653580,12,2,56600,high_risk
77532,17700.0,10.662,80900,0.629172,11,2,50900,high_risk
77533,17600.0,10.595,80300,0.626401,11,2,50300,high_risk
77534,16300.0,10.068,75300,0.601594,10,2,45300,high_risk


# Split the Data into Training and Testing

In [4]:
# Create our features
from sklearn import preprocessing
X = pd.get_dummies(df_lend.drop("loan_status", axis=1))

# Create our target
y = df_lend["loan_status"].values.reshape(-1, 1)



In [5]:
# Check the balance of our target values
from sklearn.metrics import balanced_accuracy_score
print(type(y))
y_df = pd.DataFrame(y)
print(y_df[0].value_counts())


<class 'numpy.ndarray'>
low_risk     75036
high_risk     2500
Name: 0, dtype: int64


In [6]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3, stratify=y)
X_train.shape


(58152, 10)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [7]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler

StandardScaler()

In [8]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)
X_scaler

StandardScaler()

In [9]:
# Scale the training and testing data
X_train_scale = X_scaler.transform(X_train)
X_test_scale = X_scaler.transform(X_test)
X_train_scale.shape

(58152, 10)

# Simple Logistic Regression 
Going to leave random_state at 1, due to the instructions in the other sections asking for 1.

In [40]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [41]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9528011514473053

In [42]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  569,    56],
       [   90, 18669]], dtype=int64)

In [71]:
# Print the imbalanced classification report
# The report shows the precision(pre), recall(rec), specificity(spe), geometric mean(geo),
# index balanced accuracy(iba), and support(sup)
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      1.00      0.99      0.92      1.00      0.99       625
   low_risk       1.00      0.99      1.00      1.00      1.00      0.99     18759

avg / total       1.00      0.99      1.00      0.99      1.00      0.99     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [44]:
# Resample the training data with the RandomOversampler
# Import Random Over Sampler
from imblearn.over_sampling import RandomOverSampler
random_over = RandomOverSampler(random_state=1)
X_resampl, y_resampl = random_over.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampl)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [45]:
# Train the Logistic Regression model using the resampled data
log_regr_model = LogisticRegression(solver="lbfgs", random_state=1)
log_regr_model.fit(X_resampl, y_resampl)

LogisticRegression(random_state=1)

In [46]:
# Calculate the balanced accuracy score. 
# This is to predict the positive and negative classes. This seems to be a high score.
balanced_accuracy_score(y_test, y_pred)

0.9528011514473053

In [47]:
# Display the confusion matrix
y_pred = log_regr_model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  624,     1],
       [  100, 18659]], dtype=int64)

In [48]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
y_true = [0, 1, 2, 2, 2]
y_pred = [0, 0, 2, 2, 1] 
target_names = ["class_0", "class_1", "class_2"]
print(classification_report_imbalanced(y_true, y_pred, target_names=target_names))


                   pre       rec       spe        f1       geo       iba       sup

    class_0       0.50      1.00      0.75      0.67      0.87      0.77         1
    class_1       0.00      0.00      0.75      0.00      0.00      0.00         1
    class_2       1.00      0.67      1.00      0.80      0.82      0.64         3

avg / total       0.70      0.60      0.90      0.61      0.66      0.54         5



### SMOTE Oversampling

In [49]:
# Resample training data W/SMOTE to help resolve overfitting issues affecting the minority class.
# import SMOTE
from imblearn.over_sampling import SMOTE
X_resampl, y_resampl = SMOTE(random_state=1, sampling_strategy=1).fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampl)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [50]:
# Train the Logistic Regression model using the resampled data
log_regr_model = LogisticRegression(solver="lbfgs", random_state=1)
log_regr_model.fit(X_resampl, y_resampl)

LogisticRegression(random_state=1)

In [51]:
# Calculate the balanced accuracy score
y_pred = log_regr_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9964279972279972

In [52]:
# Display the confusion matrix
y_pred = log_regr_model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  624,     1],
       [  104, 18655]], dtype=int64)

In [70]:
# Print the imbalanced classification report 
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      1.00      0.99      0.92      1.00      0.99       625
   low_risk       1.00      0.99      1.00      1.00      1.00      0.99     18759

avg / total       1.00      0.99      1.00      0.99      1.00      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [54]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
clust_centr = ClusterCentroids(random_state=1)
X_resampl, y_resampl = clust_centr.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampl)

Counter({'high_risk': 1875, 'low_risk': 1875})

In [55]:
# Train the Logistic Regression model using the resampled data
log_regr_model = LogisticRegression(solver="lbfgs", random_state=1)
log_regr_model.fit(X_resampl, y_resampl)

LogisticRegression(random_state=1)

In [56]:
# Calculate the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9528011514473053

In [57]:
# Display the confusion matrix
pd.DataFrame(
    confusion_matrix(y_test, y_pred), index=["High_risk", "Low_risk"],
    columns=["Positive", "Negative"])

,Positive,Negative
High_risk,569,56
Low_risk,90,18669


In [58]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.91      1.00      0.89      0.95      0.90       625
   low_risk       1.00      1.00      0.91      1.00      0.95      0.91     18759

avg / total       0.99      0.99      0.91      0.99      0.95      0.91     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [61]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN
smote = SMOTEENN(random_state=1)
X_resampl, y_resampl = smote.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampl)

Counter({'high_risk': 55504, 'low_risk': 55853})

In [63]:
# Train the Logistic Regression model using the resampled data
log_regr_model = LogisticRegression(solver="lbfgs", random_state=1)
log_regr_model.fit(X_resampl, y_resampl)

LogisticRegression(random_state=1)

In [65]:
# Calculate the balanced accuracy score
y_pred = log_regr_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9964013433551895

In [68]:
# Display the confusion matrix
pd.DataFrame(confusion_matrix(y_test, y_pred), index=["High_risk", "Low_risk"], 
    columns=["Positive", "Negative"])

,Positive,Negative
High_risk,624,1
Low_risk,105,18654


In [69]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      1.00      0.99      0.92      1.00      0.99       625
   low_risk       1.00      0.99      1.00      1.00      1.00      0.99     18759

avg / total       1.00      0.99      1.00      0.99      1.00      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

SMOTE Oversampling: 0.9964279972279972


2. Which model had the best recall score?

All were at 0.99 except Naive Random Oversampling

3. Which model had the best geometric mean score?

Combination (Over and Under) Sampling at 1.00
SMOTE Oversampling at 1.00
Simple Logistic Regression at 1.00

